In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(0)

In [29]:
# Example Usage:
query, key, value = torch.randn(2, 3, 8, device=device), torch.randn(2, 3, 8, device=device), torch.randn(2, 3, 8, device=device)
sdpa = F.scaled_dot_product_attention(query, key, value, attn_mask=None, is_causal=True)
print(sdpa)
print(sdpa.shape)

tensor([[[ 1.6459, -1.3602,  0.3446,  0.5199, -2.6133, -1.6965, -0.2282,
           0.2800],
         [ 0.7468, -0.4366,  0.3403,  0.4778, -0.6401, -1.1623,  0.4206,
          -0.4956],
         [ 0.5709, -0.5320,  0.3018,  0.4265, -0.3018, -0.9713,  0.3593,
          -0.5299]],

        [[-0.6227, -0.4637,  1.9218, -0.4025,  0.1239,  1.1648,  0.9234,
           1.3873],
         [-0.7674, -0.4389,  0.4084,  0.1348,  0.3939,  0.7774,  1.4936,
           0.0272],
         [-0.5494, -0.3152,  0.7227, -0.3916,  0.1765,  0.4862,  0.8529,
           0.5774]]])
torch.Size([2, 3, 8])


In [30]:
T = 3
C = 8
wei = key @ torch.transpose(query, 1, 2) * C**-0.5

In [31]:
tril = torch.tril(torch.ones(C,C))
wei = torch.masked_fill(wei, tril[:T,:T] == 0, float('-Inf')) 
wei = F.softmax(wei, dim=-1)
out = wei @ value
print(out)
print(out.shape)

tensor([[[ 1.6459e+00, -1.3602e+00,  3.4457e-01,  5.1987e-01, -2.6133e+00,
          -1.6965e+00, -2.2824e-01,  2.7995e-01],
         [ 4.6779e-01, -1.5000e-01,  3.3904e-01,  4.6474e-01, -2.7793e-02,
          -9.9659e-01,  6.2192e-01, -7.3627e-01],
         [ 5.0428e-01, -5.2094e-01,  2.9363e-01,  4.1442e-01, -1.6549e-01,
          -9.1369e-01,  3.6823e-01, -5.6291e-01]],

        [[-6.2270e-01, -4.6372e-01,  1.9218e+00, -4.0255e-01,  1.2390e-01,
           1.1648e+00,  9.2337e-01,  1.3873e+00],
         [-7.7026e-01, -4.3836e-01,  3.7838e-01,  1.4548e-01,  3.9927e-01,
           7.6974e-01,  1.5049e+00,  2.4284e-04],
         [-6.4220e-01, -2.7767e-01, -5.1086e-01, -9.0786e-03,  3.7495e-01,
           1.2069e-01,  1.2478e+00, -4.9542e-01]]])
torch.Size([2, 3, 8])


In [32]:
wei

tensor([[[1.0000, 0.0000, 0.0000],
         [0.1579, 0.8421, 0.0000],
         [0.2211, 0.6674, 0.1115]],

        [[1.0000, 0.0000, 0.0000],
         [0.4339, 0.5661, 0.0000],
         [0.0868, 0.6420, 0.2711]]])